# Data preparation

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append("../common/")

from data_utils import TRAIN_IMAGE_IDS, get_resized_polygons, generate_label_file
from image_utils import generate_pansharpened

for image_id in TRAIN_IMAGE_IDS:
    print "--", image_id
    # Generate pansharpened
    generate_pansharpened(image_id, 'ms')
    generate_pansharpened(image_id, 'swir')
    
    # generate label mask
    generate_label_file(image_id)


-- 6040_2_2
-- 6120_2_2
-- 6120_2_0
-- 6090_2_0
-- 6040_1_3
-- 6040_1_0
-- 6100_1_3
-- 6010_4_2
-- 6110_4_0
-- 6140_3_1
-- 6110_1_2
-- 6100_2_3
-- 6150_2_3
-- 6160_2_1
-- 6140_1_2
-- 6110_3_1
-- 6010_4_4
-- 6170_2_4
-- 6170_4_1
-- 6170_0_4
-- 6060_2_3
-- 6070_2_3
-- 6010_1_2
-- 6040_4_4
-- 6100_2_2
